In [ ]:
import os
import re
import time
import pytz
import asyncio
import string
import pymysql
from datetime import datetime
from dotenv import load_dotenv

from telegram import Update, BotCommand
from telegram.ext import ApplicationBuilder, MessageHandler, CommandHandler, ContextTypes, filters, ConversationHandler
#from telegram.constants import ParseMode

# Just for Jupyter
import nest_asyncio
nest_asyncio.apply()

# load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_CHARSET = os.getenv('DB_CHARSET')
BOT_TOKEN = os.getenv('BOT_TOKEN')


def get_mysql_connection():
    return pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASS, database=DB_DATABASE, charset=DB_CHARSET)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    
    con = get_mysql_connection()

    with con:
        cur = con.cursor()
        cur.execute('SELECT tg_uid, time_zone, notification_time '+
                    'FROM profiles '+
                    'WHERE tg_uid = %s;', (update.message.from_user.id, ))
        rows = cur.fetchall()

    msg = f'Приветствую {update.effective_user.first_name}!\n'
    if len(rows) == 0:
        context.user_data["user_id"] = 0
        msg += 'Для настройки предпочтений по уводомлениям выберите /settings'
    else:
        context.user_data["user_id"] = update.message.from_user.id
        msg += 'У Вас есть текущие подписки, чтобы отобразить их список выберите команду /info'

    await update.message.reply_text(msg)


async def settings(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:

    await update.message.reply_text(f'Укажите Ваш часовой пояс, например +03:00 для Москвы') #,
    return 1

        
async def get_timezone(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    tz_regexp = re.compile('^(?:Z|[+-](?:2[0-3]|[01][0-9]):[0-5][0-9])$')
    user_tz = re.search(tz_regexp, update.message.text)
    if user_tz is None:
        await update.message.reply_text(f'Пожалуйста, введите часовой пояс. Например +03:00 для Москвы или -08:00 для Купертино')
        return 1        
    else:
        user_tz = user_tz.group(0)
        if user_tz.startswith("+"):
            user_tz = user_tz[1:]
        context.user_data["user_tz"] = user_tz[1:]
        await update.message.reply_text(f'Принято, укажите лучшее время для получения уведомлений, например 14:00')
        return 2

async def finish_settings(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    time_regexp = re.compile('^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$')
    user_time = re.search(time_regexp, update.message.text)
    if user_time is None:
        await update.message.reply_text(f'Пожалуйста, введите время для уведомлений в формате ЧЧ:ММ')
        return 2
    else:
        user_time = user_time.group(0)
        user_tz = context.user_data["user_tz"]
    
        con = get_mysql_connection()

        with con:
            cur = con.cursor()
            cur.execute('INSERT INTO profiles (tg_uid, time_zone, notification_time) VALUES (%s, %s, %s) ' +
                        'ON DUPLICATE KEY UPDATE time_zone = %s, notification_time = %s;', 
                        (update.message.from_user.id, user_tz, user_time, user_tz, user_time))
            con.commit()

        await update.message.reply_text(f'Спасибо, Ваши предпочтения сохранены')
        return ConversationHandler.END
        
    
async def help(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(f'Я могу присылать уведомления по интересующим Вас темам из канала @avitotech. ' +
                                    'Для настройки уведомлений: выполните команду /subscribe и выберите интересущющие Вас темы. ' +
                                   'Для просмотра существующих подписок выполните команду /info')

    
async def info(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:

    con = get_mysql_connection()
    
    with con:
        cur = con.cursor()
        cur.execute('SELECT n.keyword_id, h.hashtag FROM `notifications` n ' +
                    'JOIN hashtags h ON h.hashtag_id = n.keyword_id ' +
                    'WHERE n.tg_uid = %s;', (update.message.from_user.id, ))
        rows = cur.fetchall()

    msg = f'У Вас нет ни одной подписки'
    if len(rows) > 0:
        msg = 'Ваши активные подписки:\n\n'
        for row in rows:
            msg += f'{row[0]}. #{row[1]}\n'
            
        msg += '\nИспользуйте /subscribe, чтобы добавить подписки к списку, либо /clear, чтобы очистить список подписок'

    await update.message.reply_text(msg)
        

async def clear(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    
    con = get_mysql_connection()
    
    with con:
        cur = con.cursor()
        cur.execute('DELETE FROM notifications WHERE tg_uid = %s;', (update.message.from_user.id, ))
        con.commit()                    
    
    await update.message.reply_text(f'Список подписок очищен', parse_mode='html')

    
async def subscribe(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    con = get_mysql_connection()
    
    with con:
        cur = con.cursor()
        cur.execute('SELECT hashtag_id, hashtag ' + 
                    'FROM hashtags ')
        rows = cur.fetchall()

    if len(rows) > 0:
        msg = 'Полный лист подписок:\n\n'
        for row in rows:
            msg += f'{row[0]}. #{row[1]}\n'
            
        msg += '\nВведите через запятую номера подписок, которые Вы хотели бы отслеживать'

        await update.message.reply_text(msg)
        
        return 1

    
async def finish_subscribe(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    subscribe_regexp = re.compile('[0-9 ,]')
    user_subscribe = re.search(subscribe_regexp, update.message.text)
    print(user_subscribe.group(0))

    if user_subscribe is None:
        await update.message.reply_text(f'Пожалуйста, введите номера подписок через запятую')
        return 1
    else:
        con = get_mysql_connection()
        
        a = ''.join(filter(lambda x: x in string.printable, user_subscribe.group(0)))

        with con:
            cur = con.cursor()
            if ',' not in a:
                if a.isdigit():
                    cur.execute('INSERT IGNORE INTO `notifications`' +
                                'SET `tg_uid` = %s, `keyword_id` = %s',
                                (update.message.from_user.id, a))
            else:
                for hashtag_n in a.split(','):
                    if hashtag_n.isdigit():
                        cur.execute('INSERT IGNORE INTO `notifications`' +
                                    'SET `tg_uid` = %s, `keyword_id` = %s',
                                    (update.message.from_user.id, hashtag_n))
            con.commit()

        await update.message.reply_text(f'Спасибо, Ваши предпочтения сохранены')
        return ConversationHandler.END

    
async def new_hashtags(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    
    # TODO:
    # After being added to the channel, the bot monitors new posts and store hashtags from those
    # if update.message.chat_id == 'AVITO.TECH CHANNEL ID'...
    
    a = ''.join(filter(lambda x: x in string.printable, update.message.text))

    con = get_mysql_connection()
    
    with con:
        for hashtag in a.split(' '):
            if hashtag.startswith('#'):
                cur = con.cursor()
                cur.execute('INSERT IGNORE INTO hashtags ' +
                            'SET hashtag = %s ', 
                            (hashtag.translate(str.maketrans('', '', string.punctuation))))
        con.commit()


async def chat(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    assistand_answer = 'Извините, я затрудняюсь ответить на этот вопрос. Для помощи наберите /help'
    await update.message.reply_text(assistand_answer) #, parse_mode='html'

    
async def main() -> None:
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("info", info))
    app.add_handler(CommandHandler("clear", clear))
    app.add_handler(CommandHandler("help", help))
    app.add_handler(CommandHandler("new_hashtags", new_hashtags))

    handler = ConversationHandler(
        allow_reentry=True,
        entry_points=[CommandHandler('settings', settings)],
        states={
            1: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_timezone)],
            2: [MessageHandler(filters.TEXT & ~filters.COMMAND, finish_settings)]
            
        },
        fallbacks=[]
    )
    app.add_handler(handler)    

    handler = ConversationHandler(
        allow_reentry=True,
        entry_points=[CommandHandler('subscribe', subscribe)],
        states={
            1: [MessageHandler(filters.TEXT & ~filters.COMMAND, finish_subscribe)],
            
        },
        fallbacks=[]
    )
    app.add_handler(handler)    

    
    app.add_handler(MessageHandler(filters.TEXT, chat))

    
    bot_commands = [
        BotCommand(command="/start", description="Запуск бота"),
        BotCommand(command="/settings", description="Общие настройки"),
        BotCommand(command="/info", description="Cписок подписок"),
        BotCommand(command="/subscribe", description="Добавление подписки"),        
        BotCommand(command="/clear", description="Очистить список"),
        BotCommand(command="/help", description="Информация"),
    ]
    await app.bot.set_my_commands(bot_commands)

    app.run_polling()


if __name__ == "__main__":
    await main()

